# Семинар 8: полезности и одна задача

### Вопросики

In [126]:
a = [1, 2, -5, -10]
sorted(a)

[-10, -5, 1, 2]

In [127]:
sorted(a, reverse=True)

[2, 1, -5, -10]

In [128]:
a = ['Маша', "Вася", 'Яна']
sorted(a)

['Вася', 'Маша', 'Яна']

In [129]:
sorted(a, key=lambda w: w[-1])

['Маша', 'Яна', 'Вася']

In [130]:
x = """njlgljnge
lnfwlewgln
nfewlnwfelnk
lknfwlnkdeglk"""

with open('file.txt', 'w') as f:
    f.write(x)

In [131]:
with open('file.txt', 'r') as f:
    b = f.readlines()
    
print(''.join(b))

njlgljnge
lnfwlewgln
nfewlnwfelnk
lknfwlnkdeglk


## Задача

В файлике `markup.json` лежит разметка новостных статей на четыре категории. Разметку делали [в толоке.](https://toloka.yandex.ru/tasks)

```
{'inputValues': {'id': 1, 'version': 1},
 'workerId': 465,
 'taskId': 0,
 'outputValues': {'result': 'ok'}}
```

- в поле `inputValues` лежит `id` документа и его версия
- в поле `workerId` лежит `id` модератора, который делал разметку
- в поле `taskId` лежит номер таски, в которой модератору показалась новость (в одной таске могут быть несколько новостей)
- в поле `outputValues` лежат результаты разметки

In [132]:
import json

with open("sem07_211/markup.json", "r") as f:
    x = json.load(f)    
x[:5]

[{'inputValues': {'id': 1, 'version': 1},
  'workerId': 465,
  'taskId': 0,
  'outputValues': {'result': 'ok'}},
 {'inputValues': {'id': 1, 'version': 1},
  'workerId': 365,
  'taskId': 0,
  'outputValues': {'result': 'ok'}},
 {'inputValues': {'id': 1, 'version': 1},
  'workerId': 619,
  'taskId': 0,
  'outputValues': {'result': 'local'}},
 {'inputValues': {'id': 2, 'version': 1},
  'workerId': 759,
  'taskId': 0,
  'outputValues': {'result': 'ok'}},
 {'inputValues': {'id': 2, 'version': 1},
  'workerId': 694,
  'taskId': 0,
  'outputValues': {'result': 'ok'}}]

__Задание 1__ Ответьте по данным на следующие вопросы:

__а)__ Сколько всего уникальных новостей было размечено?

In [28]:
# your code here ( ⚆ _ ⚆)
len(set([item['inputValues']['id'] for item in x]))

1000

__б)__ Сколько всего есть уникальных категорий в поле `result`? Как они называются? 

In [40]:
# your code here ( ⚆ ω ⚆)
from collections import Counter

cnt = Counter([item['outputValues']['result'] for item in x])
cnt

Counter({'ok': 1448, 'local': 543, 'violence': 508, 'federal': 501})

In [41]:
cnt.most_common()

[('ok', 1448), ('local', 543), ('violence', 508), ('federal', 501)]

__в)__ Сколько человек занимались разметкой?

In [51]:
# your code here  ( ͡° ͜ʖ ͡°)
len(set([item['workerId'] for item in x]))

957

In [52]:
len(x)

3000

__г)__ Сколько новостей в среднем приходится на один `taskId`? 

In [63]:
# your code here  ლ(ಠ益ಠლ)
cnt = Counter([item['taskId'] for item in x])
taskId_cnt = cnt.values()
sum(taskId_cnt)/len(taskId_cnt)

8.982035928143713

Обратите внимание, что все генераторы ленивые и по ним можно ходить только один раз!

In [111]:
from itertools import groupby

news_by_typeId = map(
    lambda w: len(list(w[1])),
        groupby(
            sorted(x, key=lambda w: w['taskId']),
            key=lambda w: w['taskId']
        )
    )

news_by_typeId = list(news_by_typeId)
sum(news_by_typeId)/len(news_by_typeId)

8.982035928143713

In [ ]:
# Если не делать лист работать не будет! 
# Посчитает только первый вызов и поделит на ноль

# news_by_typeId = list(news_by_typeId)
sum(news_by_typeId)/len(news_by_typeId)

__д)__ Правда ли, что все входные поля имеют структуру описанную выше? Напишите функцию, которая будет проверять консистентность входных данных. Если данные имеют другую структуру, функция должна вызывать исключение типа `ValueError`.

{'inputValues': {'id': 1, 'version': 1},
 'workerId': 465,
 'taskId': 0,
 'outputValues': {'result': 'ok'}}

In [135]:
cn = ('inputValues' in x[0])
cn

True

In [136]:
len(x[0])

4

In [137]:
isinstance(4, int)

True

In [139]:
isinstance('4', int)

False

In [150]:
def check_format(item):
    # Можно чекнуть что есть ключи 
    # Можно чекнуть что значения инты а не дичь
    # Можно чекнуть что нет лишних полей
    
    cond_input = [
        ('inputValues' in item),
        (item.get('workerId') is not None),
        (item.get('taskId') is not None),
        ('outputValues' in item),
        len(item) == 4,
        isinstance(item.get('workerId'), int)
    ]
    
    if not all(cond_input):
        raise ValueError('Что-то не так с входными данными') 
    else:
        return item

In [151]:
check_format(x[5])

{'inputValues': {'id': 2, 'version': 1},
 'workerId': 182,
 'taskId': 0,
 'outputValues': {'result': 'ok'}}

In [152]:
check_format(
    {'inputValues': {'id': 2, 'version': 1},
 'outputValues': {'result': 'ok'}}
)

ValueError: Что-то не так с входными данными

__Задание 2__ 

Каждая новость даётся на разметку нескольким модераторам. Мы хотим агрегировать разметку до одного числа. Напишите функцию `aggregate`, которая будет делать это методом большинства. 

На выходе мы должны получить json вида: 

```
{'inputValues': {'id': 1, 'version': 1},
 'workers': [465, 10, 55],
 'taskIds': [0, 1, 18],
 'rawnumber': 3,
 'probability': 0.66,
 'algorithm': 'majority_vote',
 'outputValues': {'result': 'ok'}}
```

В поле `rawNumber` записано количество ответов модераторов. В поле `probability` доля ответов за больший класс (majority vote).

Если все три модератора проголосовали разные вердикты, выбирайте рандомный.

In [159]:
from operator import itemgetter as at
from itertools import groupby

In [189]:
def aggregate(assigments):
    
    agg_x = []
    
    for item in assigments:
        item['key'] = (item['inputValues']['id'], item['inputValues']['version'])
    
    cnt = 0
    for k, item in groupby(sorted(assigments, key=at('key')), key=at('key')):
        
        aggregated = { }

        ret = at('taskId', 'workerId', 'inputValues', 'outputValues')
        taskIds, workerIds, inp, output =  zip(*map(ret, item))

        aggregated['workerIds'] = list(workerIds)
        aggregated['taskIds'] = list(taskIds)
        aggregated['inputValues'] = inp[0]
        aggregated['outputValues'] = {'result': [at('result')(jtem) for jtem in output]}
        agg_x.append(aggregated)
    return  agg_x

In [181]:
jtem = [{'inputValues': {'id': 1, 'version': 1}, 'workerId': 465, 'taskId': 0, 'outputValues': {'result': 'ok'}, 'key': (1, 1)}, {'inputValues': {'id': 1, 'version': 1}, 'workerId': 365, 'taskId': 0, 'outputValues': {'result': 'ok'}, 'key': (1, 1)}, {'inputValues': {'id': 1, 'version': 1}, 'workerId': 619, 'taskId': 0, 'outputValues': {'result': 'local'}, 'key': (1, 1)}]

In [182]:
ret = at('taskId', 'workerId', 'inputValues', 'outputValues')

In [185]:
list(map(ret, jtem))

[(0, 465, {'id': 1, 'version': 1}, {'result': 'ok'}),
 (0, 365, {'id': 1, 'version': 1}, {'result': 'ok'}),
 (0, 619, {'id': 1, 'version': 1}, {'result': 'local'})]

In [187]:
taskIds, workerIds, inp, output = zip(*map(ret, jtem))
taskIds

(0, 0, 0)

In [188]:
workerIds

(465, 365, 619)

In [190]:
inp

({'id': 1, 'version': 1}, {'id': 1, 'version': 1}, {'id': 1, 'version': 1})

In [192]:
output

({'result': 'ok'}, {'result': 'ok'}, {'result': 'local'})

In [193]:
aggregated = aggregate(x)
aggregated[:2]

[{'workerIds': [465, 365, 619],
  'taskIds': [0, 0, 0],
  'inputValues': {'id': 1, 'version': 1},
  'outputValues': {'result': ['ok', 'ok', 'local']}},
 {'workerIds': [759, 694, 182],
  'taskIds': [0, 0, 0],
  'inputValues': {'id': 2, 'version': 1},
  'outputValues': {'result': ['ok', 'ok', 'ok']}}]

In [ ]:

agg = aggregate(x)
len(agg)

In [ ]:
# your code here  (ﾉ´ з `)ノ

def simple_mv(output):
    pass


__Задание 3__

В данных иногда встречаются ситуации, когда три модератора сказали разные вердикты. Не очень понятно что делать с разметкой в таком случае. Давайте будем обрабатывать такие краевые ситуации с помощью какого-нибудь бизнес-правила.

Пусть на множестве наших вердикто задан порядок. Я хочу реализовать два правила: 

__1)__ брать самый жёсткий из выставленных

__2)__ брать взвешенный вердикт



In [4]:
WEIGHTS = {
    'ok': 0,
    'violence': 1,
    'local': 0.5,
    'federal': 0
}

AGG_METHOD = 'max'

In [5]:
def get_most_dangerous(output, w=WEIGHTS):
    pass

In [6]:
def get_average(output, w=WEIGHTS):
    pass

In [7]:
def get_verdict(output, w=WEIGHTS):
    mv_verd, prob = simple_mv(output)
    if prob > 0.5:
        return mv_verd
    else:
        if AGG_METHOD == 'max':
            raise NotImplementedError
        elif AGG_METHOD == 'avg':
            raise NotImplementedError
        elif AGG_METHOD == 'random':
            raise NotImplementedError
        else:
            raise NotImplementedError

# Справочная информация

## 1. Исключения

In [113]:
x = [7, 8, 9, 0, 4, -5]

for item in x:
    print(42 / item)

6.0
5.25
4.666666666666667


ZeroDivisionError: division by zero

In [114]:
x = [7, 8, 9, 0, 4, -5]

for item in x:
    try:
        print(42 / item)
    except:
        print('Не могу делить, начальник!')

6.0
5.25
4.666666666666667
Не могу делить, начальник!
10.5
-8.4


In [115]:
from math import log

x = [7, 8, 9, 0, 4, -5]

for item in x:
    try:
        print(42 / log(item + 1))
    except ZeroDivisionError:
        print('Не могу делить, начальник!')

20.19773057244549
19.115023759163584
18.240368239936576
Не могу делить, начальник!
26.096067251503698


ValueError: math domain error

In [116]:
from math import log

x = [7, 8, 9, 0, 4, -5]

for item in x:
    try:
        print(42 / log(item + 1))
    except ZeroDivisionError:
        print('Не могу делить, начальник!')
    except ValueError:
        print('Не могу искать лоагрифм, начальник!')

20.19773057244549
19.115023759163584
18.240368239936576
Не могу делить, начальник!
26.096067251503698
Не могу искать лоагрифм, начальник!


In [117]:
x = 4
assert x < 5, 'На самом деле это не так'

In [118]:
x = 15
assert x < 5, 'На самом деле это не так'

AssertionError: На самом деле это не так

In [120]:
def f(x):
    if x > 5:
        raise ValueError('На вход пришли плохие данные, посмотри на них!')
    else:
        return x

In [121]:
f(3)

3

In [122]:
f(17)

ValueError: На вход пришли плохие данные, посмотри на них!

Можно отлавливать конкретные ошибки и записывать их в массив!

In [123]:
errors = [ ]

for x in [4, 5, 17, 22, 0, 5 , 8, 2]:
    try:
        print(f(x))
    except ValueError:
        errors.append(x)
        print('Возникла ошибка и я запомнил данные где она произошла')

4
5
Возникла ошибка и я запомнил данные где она произошла
Возникла ошибка и я запомнил данные где она произошла
0
5
Возникла ошибка и я запомнил данные где она произошла
2


In [124]:
errors

[17, 22, 8]

## 2. List comprehension

Я хочу посчитать сумму первых ста чисел.



In [29]:
n = 100
s = 0

for i in range(1, n + 1):
    s += i
s

5050

In [30]:
[i for i in range(10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [31]:
sum([i for i in range(1, 101)])

5050

Я хочу посчитать сумму квадротов первых ста чисел если они четные

In [32]:
[i ** 2 for i in range(10) if i % 2 == 0]

[0, 4, 16, 36, 64]

In [33]:
sum([i ** 2 for i in range(1, 101) if i % 2 == 0])

171700

In [34]:
# Тернарные выражения
a = 6

ans = 'even' if a % 2 == 0  else 'odd'
ans

'even'

In [35]:
['even' if i % 2 == 0  else 'odd'  for i in range(10)]

['even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd']

In [36]:
[[i + j for i in range(5)] for j in range(5)]

[[0, 1, 2, 3, 4],
 [1, 2, 3, 4, 5],
 [2, 3, 4, 5, 6],
 [3, 4, 5, 6, 7],
 [4, 5, 6, 7, 8]]

## 3. Полезные мелочи

In [ ]:
# все внутри должно быть True
all([True, False, True])

In [ ]:
# хотябы 1 объект нутри должно быть True
any([True, False, True])

In [ ]:
a = [1, 2, 3, 4, 5]

fun = lambda w: w ** 2

map(fun, a) # обещание применить функцию fun к каждому объекту из a

Оставьте в тексте ниже только цифры

In [ ]:
text = "Все 4 ваши 8 проб15лемы реш17ит ко23д 42"

In [ ]:
''.join([letter for letter in text if letter.isdigit()])

In [ ]:
list(map(lambda w: w.isdigit(), text))

In [ ]:
''.join(filter(lambda w: w.isdigit(), text))

## 4. Модуль collections

Внутри этого модуля куча полезных структур данных.

In [42]:
from collections import Counter

a = ['red', 'blue', 'red', 'green', 'blue', 'blue']
cnt = Counter(a)
cnt

Counter({'red': 2, 'blue': 3, 'green': 1})

In [43]:
cnt['red']

2

In [44]:
cnt.most_common()

[('blue', 3), ('red', 2), ('green', 1)]

Для удобной работы со словарями есть `defaultdict`

In [45]:
sentence = "The red for jumped over the fence and ran to the zoo for food"
words = sentence.split(' ')
 
reg_dict = {}
for word in words:
    if word in reg_dict:
        reg_dict[word] += 1
    else:
        reg_dict[word] = 1

reg_dict

{'The': 1,
 'red': 1,
 'for': 2,
 'jumped': 1,
 'over': 1,
 'the': 2,
 'fence': 1,
 'and': 1,
 'ran': 1,
 'to': 1,
 'zoo': 1,
 'food': 1}

In [49]:
from collections import defaultdict

sentence = "The red for jumped over the fence and ran to the zoo for food"
words = sentence.split(' ')
 
reg_dict = defaultdict(lambda: 0)
for word in words:
    reg_dict[word] += 1

dict(reg_dict)

{'The': 1,
 'red': 1,
 'for': 2,
 'jumped': 1,
 'over': 1,
 'the': 2,
 'fence': 1,
 'and': 1,
 'ran': 1,
 'to': 1,
 'zoo': 1,
 'food': 1}

- [документация](https://docs.python.org/3.9/library/collections.html?highlight=collections#module-collections)
- [примеры использования](https://python-scripts.com/import-collections)

## 5. Модуль itertools

Внутри этого модуля куча полезных итераторов.

Внутри набор быстрых и эффективных по памяти иснтументов. Вместе формируют "алгебру итераторов", которая позволяет лаконично и эффективно создвать на python разные штуки. Многие очень специфичные, но рано или поздно могут пригодиться. Рекомендую почитать что есть внутри.

In [64]:
from itertools import permutations

# Обещание сгенерировать все перестановки (генератор)
permutations([1, 2, 3])

In [65]:
list(permutations([1, 2, 3]))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [66]:
list(permutations(['Вика', 'Tom', 'Alex']))

[('Вика', 'Tom', 'Alex'),
 ('Вика', 'Alex', 'Tom'),
 ('Tom', 'Вика', 'Alex'),
 ('Tom', 'Alex', 'Вика'),
 ('Alex', 'Вика', 'Tom'),
 ('Alex', 'Tom', 'Вика')]

Другое полезный пример - это groupby!

In [77]:
from itertools import groupby

student = [
        ('john', 'A', 15),
        ('jane', 'B', 12),
        ('daavee', 'B', 10),
    ]

По ключу `key` можно указать функцию, которая вычисляется при работе функции.

In [78]:
def f(w):
    return w[-1]
    
max(student, key=f)

('john', 'A', 15)

In [79]:
max(student, key=lambda w: w[-1])

('john', 'A', 15)

In [80]:
max(student, key=lambda w: len(w[0]))

('daavee', 'B', 10)

In [81]:
sorted(student, key=lambda w: w[-1])

[('daavee', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

In [82]:
groupby(student, key=lambda w: w[1])

In [83]:
list( groupby(student, key=lambda w: w[1]))

[('A', <itertools._grouper at 0x7f52d755fe80>),
 ('B', <itertools._grouper at 0x7f52d755f6d8>)]

In [84]:
[(i, list(v)) for i,v in groupby(student, key=lambda w: w[1]) ]

[('A', [('john', 'A', 15)]), ('B', [('jane', 'B', 12), ('daavee', 'B', 10)])]

__Важно:__ этот `groupby` глуповат и умеет группировать только отсротированные данные.

In [85]:
student = [
        ('jane', 'B', 12),
        ('john', 'A', 15),
        ('daavee', 'B', 10),
    ]

[(i, list(v)) for i,v in groupby(student, key=lambda w: w[1]) ]

[('B', [('jane', 'B', 12)]),
 ('A', [('john', 'A', 15)]),
 ('B', [('daavee', 'B', 10)])]

In [86]:
[(i, list(v)) for i,v in groupby(sorted(student, key=lambda w: w[1]), key=lambda w: w[1]) ]

[('A', [('john', 'A', 15)]), ('B', [('jane', 'B', 12), ('daavee', 'B', 10)])]

- [Документация](https://docs.python.org/3.9/library/itertools.html?highlight=itertools)
- [Обзорная статья на хабре с примерами кода](https://habr.com/ru/company/otus/blog/529356/)

## 6. Модуль operator

Внутри разные стандартные штуки вроде сложения, умножения и тп, но реализованные эффективно по памяти. Этот модуль очень удобно совмещать с itertools, код работает быстрее.

Одна из самых необычных штук - концепция геттеров. Это функция, которая извлекает по указанному пути нужные части структуры. Они полезны при работе с итераторами или генераторами последовательностей, так как вычисляются эффективнее, чем лямбда-функции, записанные в key.

In [153]:
from operator import itemgetter as at

student = [
        ('john', 'A', 15),
        ('jane', 'B', 12),
        ('daavee', 'B', 10),
    ]

# указал 2 координату
# lambda w: w[2]
sorted(student, key=at(2))

[('daavee', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

In [154]:
# указал 1 координату
sorted(student, key=at(1))

[('john', 'A', 15), ('jane', 'B', 12), ('daavee', 'B', 10)]

In [156]:
# сортируем сначала по оценке, а потом и по возрасту
sorted(student, key=at(2,1))

[('daavee', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

In [157]:
# распаковка groupby
[(i, list(v)) for i,v in groupby(student, key=at(1)) ]

[('A', [('john', 'A', 15)]), ('B', [('jane', 'B', 12), ('daavee', 'B', 10)])]

Можно заставлять его вытаскивать конкретные ключи, а не координаты!



In [158]:
adict = [{'a': 1}, {'a': 1}, {'a': 5}]

[(i, list(v)) for i,v in groupby(adict, at('a'))]

[(1, [{'a': 1}, {'a': 1}]), (5, [{'a': 5}])]

- [Документация](https://docs.python.org/3.9/library/operator.html)
- [Примеры использования, но сложноватые](https://pymotw.com/2/operator/)